# Collective Loan in Marlowe


**Executive Summary**

This collective-loan contract lets lenders deposit currency such as `DjedUSD` in exchange for liquidity tokens, and then lets borrowers draw funds from the contract as loans. After the borrowers pay back their borrowings with interest to the collective loan, the lenders then redeem their liquidity tokens for `DjedUSD` that includes profit from the lending. A central administrator manages approvals. The Marlowe contract for this collective loan is scalable to an arbitrarily largenumber of lenders or borrowers, so long as the trusted central administrator is relied upon to disperse role tokens on a just-in-time basis. (Less centralized versions may be feasible, but they would be more rigid in terms of sequence of participation of the lenders and borrowers.) The contract was both simulated and run on the Cardano blockchain: that involved 30 lenders, 70 borrowers, and ~530 transactions.

[A simple one-lender, one-borrower version of this contract](collective-loan.blockly) is available for use in [Marlowe Playground's Blockly/Marlowe format](https://marlowe-playground-staging.plutus.aws.iohkdev.io/#/marlowe).

See also [this video](https://vimeo.com/735889853/10601168e4) that walks through this example in Marlowe Playground and on the Marlowe testnet.


**Highlights**

*   Arbitrary scalability for Marlowe contracts where role-tokens are dispersed on a just-in-time basis.
*   Incremental merkleization for rapid construction of large Marlowe contracts.
*   Novel merkleization of timeouts to limit contract depth.


***Life cycle of this collective loan:***

1.  The administrator configures and creates the contract.
2.  The administrator deposits the liquidity tokens for the contract.
3.  Lenders request to deposit their investment funds.
    1.  The administrator approves the lender.
    2.  The administrator sends a role token to the lender.
    3.  The lender deposits their funds.
    4.  The lender receives liquidity tokens as a receipt.
4.  Borrowers requests loans from the collective loan.
    1.  The administrator authorizes the loan.
    2.  The administrator sends a role token to the borrower.
    3.  The borrower withdraws their funds.
5.  Borrowers deposit their loan proceeds.
6.  Lenders redeem their liquidity tokens.
7.  The administrator closes the contract.


***Caveats and limitations***

-   The contract has a finite life cycle with phases of investment, lending, repayment, and redemption.
-   The administrator may be an organization or an oracle, manual or automated, but it must be highly secure.
    -   For example, a contract with 100 lenders and 5000 loans will fit-on chain if it is merkleized: such a contract contains 20,601 small continuations that account for all possible logical paths through the contract.
    -   However, we have not studied how the arbitrary *interleaving* of lending and payback may affect scalability.
-   Unlike fully decentralized Marlowe contracts, this collective-loan contract relies upon centralized management of role tokens to bypass the Marlowe's limitations on the number of state variables (which are stored in Plutus `Datum`).
    -   The use of reference scripts (CIP-33) in the Babbage Era may somewhat alleviate this limitation.
    -   Role tokens can be pre-distributed *if lenders and borrowers submit transactions in a pre-defined order*: this will decrease centralization and increase security at the expense of making the contract more rigid.

## Sequence Diagram

![UML sequence diagram for collective loan](collective-loan.png)

## Preliminaries

### Versions

Make a note of the versions of the Marlowe repository and software.

In [1]:
marlowe-cli --version

marlowe-cli 0.0.5.0


In [2]:
cardano-cli --version

cardano-cli 1.35.2 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [3]:
git rev-parse HEAD

f5a0f64d8697dd7a29af91b11530039dc98a1d44


### Utility Functions

The following functions simplify UTxO management.

In [4]:
function find_role_token() {
  marlowe-cli util select --asset-only $ROLES_POLICY.$2 $1 | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/'
}

In [5]:
function find_ada() {
  marlowe-cli util select --lovelace-only $((10 * ADA)) $1 | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/' | head -n 1
}

In [6]:
function find_token() {
  marlowe-cli util select --asset-only $2 $1 | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/' | head -n 1
}

In [7]:
function find_token_2() {
  marlowe-cli util select --asset-only $2 $1 | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/' | tail -n +2 | head -n 1
}

In [8]:
function make_role_txout () {
  echo "$1+$MIN_ADA+1 $ROLES_POLICY.$2"
}

In [9]:
function make_token_txout() {
  echo "$1+$MIN_ADA+$3 $2"
}

### Select the Cardano network

In [10]:
export CARDANO_TESTNET_MAGIC=1567
export CARDANO_NODE_SOCKET_PATH=node.socket

### Current Time

We'll start the contract at the current time.

In [11]:
NOW=$(($(date -u +%s) * 1000))
echo $NOW

1660498029000


Compute time constants, for convenience when setting time intervals for transactions.

In [12]:
MINUTE=$((60 * 1000))
HOUR=$((60 * MINUTE))
DAY=$((24 * HOUR))

There is a minimum Ada requirement associated with a native token.

In [13]:
ADA=1000000
MIN_ADA=$((2 * ADA))
echo $MIN_ADA

2000000


### Contract Parameters

Set parameters for the contract. This contract scales to large numbers of lenders and borrowers, but that will involve many transactions and much time if run on-chain.

In [14]:
N_LENDERS=30
N_BORROWERS=70

MINIMUM_INVESTMENT=500000
MAXIMUM_INVESTMENT=5000000

POOL_SIZE=$((N_LENDERS * MAXIMUM_INVESTMENT))

MINIMUM_LOAN=100000
MAXIMUM_LOAN=1000000
BORROWER_DJED=500000

TOTAL_DJED=$((POOL_SIZE + N_BORROWERS * BORROWER_DJED))

SETUP_DEADLINE=$((NOW + 1 * HOUR))

INVESTMENT_DEADLINE=$((NOW + 3 * HOUR))
INVESTMENT_REPAYMENT_DEADLINE=$((NOW + 12 * HOUR))

LOAN_DEADLINE=$((NOW + 6 * HOUR))
LOAN_REPAYMENT_DEADLINE=$((NOW + 9 * HOUR))

ULTIMATE_DEADLINE=$((NOW + 15 * HOUR))

### Keys for the Administrator

Use a pre-existing key for payment.

In [15]:
ADMIN_ROLE=A
ADMIN_SKEY=christopher-marlowe.skey
ADMIN_VKEY=christopher-marlowe.vkey
ADMIN_ADDR=$(cat christopher-marlowe.testnet.address)
echo $ADMIN_ADDR

addr_test1vrssw4edcts00kk6lp7p5n64666m23tpprqaarmdwkaq69gfvqnpz


Send 25 Ada to the administrator for each lender and borrower, just to make sure there is sufficient Ada to cover transaction fees.

In [16]:
marlowe-cli util faucet --lovelace $((25 * (N_LENDERS + N_BORROWERS) * ADA)) --out-file /dev/null --submit 600 $ADMIN_ADDR

TxId "e35b3bce34ab586d6456dd55af940a794dabace6503536d0ad4ad9d2cb9edb57"


### Key Derivation

We'll derive all of the keys for lenders and borrowers from a single root private key.

In [17]:
ROOT_PRV=william-shakespeare.root.prv

For the convenience of viewing transactions in Daedalus, we'll include the same stake credential in all of the addresses.

In [18]:
cardano-wallet key child 1852H/1815H/0H/2/0 < $ROOT_PRV \
| cardano-cli key convert-cardano-address-key --shelley-stake-key --signing-key-file /dev/stdin --out-file /dev/stdout \
| cardano-cli key verification-key --signing-key-file /dev/stdin --verification-key-file /dev/stdout \
| cardano-cli key non-extended-key --extended-verification-key-file /dev/stdin --verification-key-file william-shakespeare.stake.vkey
cardano-cli stake-address build --testnet-magic $CARDANO_TESTNET_MAGIC --stake-verification-key-file william-shakespeare.stake.vkey

stake_test1urplvp2a7dythh6yxutd0qlzzkncr3gy5ftvxj02d3etafqjugc5h


### Keys for Lenders

Build the signing keys and payment addresses for the lenders from the `1852H/1815H/0H/0/` sequence of key derivations.

In [19]:
mkdir -p tmp
for j in `seq 1 $N_LENDERS`
do
  LENDER_SKEY[$j]=tmp/lender-$j.skey
  LENDER_ADDR[$j]=$(
  cardano-wallet key child 1852H/1815H/0H/0/$j < $ROOT_PRV \
  | cardano-cli key convert-cardano-address-key --shelley-payment-key --signing-key-file /dev/stdin --out-file ${LENDER_SKEY[$j]}
  cardano-cli key verification-key --signing-key-file ${LENDER_SKEY[$j]} --verification-key-file /dev/stdout \
  | cardano-cli address build --testnet-magic $CARDANO_TESTNET_MAGIC --payment-verification-key-file /dev/stdin --stake-verification-key-file william-shakespeare.stake.vkey \
  )
  echo "Lender 1852H/1815H/0H/0/$j = ${LENDER_ADDR[$j]}"
done

Lender 1852H/1815H/0H/0/1 = addr_test1qzenwj7elwatk3mmc368mwnv067fqnuk3x7u4nqw5dezg8wr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqf6k36p
Lender 1852H/1815H/0H/0/2 = addr_test1qraxynufcduk7ak7nke2jm6pnc32vn7wsmm7ml30cutfgg7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq77wfgp
Lender 1852H/1815H/0H/0/3 = addr_test1qzlwepx5u26a8w94kfrkfem6njlam22emyhjscu0lxqrce7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq5jnrn3
Lender 1852H/1815H/0H/0/4 = addr_test1qqjrjfslev50d5jn6qdeztxsqvuaaefq6e4hu55n59grjj7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqwhulyr
Lender 1852H/1815H/0H/0/5 = addr_test1qqxrzfv0v8l0lahyc5qsgpug66dafp8njl278g0pter7e3xr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqcjuuw0
Lender 1852H/1815H/0H/0/6 = addr_test1qqqfzyuqkvrr7ajgk887sqac8lgzatc6el9uxke4x5g5dg7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqkmrpwu
Lender 1852H/1815H/0H/0/7 = addr_test1qp77epd4gy5hrt8pnh6h73se4durwrrkes3u0vj6tycwsq7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq7kyfpt
Lender 1852H/1815H/0H/0/8 = addr_test1qqa

Send 25 ADA to each lender.

In [20]:
marlowe-cli util faucet --lovelace $((25 * ADA)) --out-file /dev/null --submit 600 ${LENDER_ADDR[@]}

TxId "ca6a39e1af4030df5cba0f3ea99b083198c93bf8f1c2c29ffebe3e18e48d811e"


### Keys for Borrowers

Build the signing keys and payment addresses for the borrowers from the `1852H/1815H/0H/1/` sequence of key derivations.

In [21]:
mkdir -p tmp
for j in `seq 1 $N_BORROWERS`
do
  BORROWER_SKEY[$j]=tmp/borrower-$j.skey
  BORROWER_ADDR[$j]=$(
    cardano-wallet key child 1852H/1815H/0H/1/$j < $ROOT_PRV \
  | cardano-cli key convert-cardano-address-key --shelley-payment-key --signing-key-file /dev/stdin --out-file ${BORROWER_SKEY[$j]}
  cardano-cli key verification-key --signing-key-file ${BORROWER_SKEY[$j]} --verification-key-file /dev/stdout \
  | cardano-cli address build --testnet-magic $CARDANO_TESTNET_MAGIC --payment-verification-key-file /dev/stdin --stake-verification-key-file william-shakespeare.stake.vkey \
  )
  echo "Borrower 1852H/1815H/0H/1/$j = ${BORROWER_ADDR[$j]}"
done

Borrower 1852H/1815H/0H/1/1 = addr_test1qp7q4e9nlhq08mrugs8pm5uq045e425w72mxhxupq9qqgykr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqjmt3lh
Borrower 1852H/1815H/0H/1/2 = addr_test1qpzun0s6thsyf9qt0zyfnn50x8kpwutrv20eq8dgx9gt6tkr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqfsjdsz
Borrower 1852H/1815H/0H/1/3 = addr_test1qqpet7s339suufnxmr4vrg09469v30g59yxa70yrkxvrfekr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq37csh4
Borrower 1852H/1815H/0H/1/4 = addr_test1qppr8ue92zznn2k3v235fuxtpcun9k32g8jn09dsq3ndvawr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqh2lydz
Borrower 1852H/1815H/0H/1/5 = addr_test1qzcee8xm57tuss5wm3yvxhpk2w7ws3h03c8w2f30tlqahhkr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqgtpxaq
Borrower 1852H/1815H/0H/1/6 = addr_test1qr53gduqpnz38lhkmgq29axxqhttxk653fp7jxhz0c0wpekr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq3ghaj6
Borrower 1852H/1815H/0H/1/7 = addr_test1qzfturx55xqdm3pu3vwsrvvz26syajjlmpz0whxvll7cw27r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq3j88jz
Borrower 1852H/1815H/0H/1/8

Send 25 ADA to each borrower.

In [22]:
marlowe-cli util faucet --lovelace $((25 * ADA)) --out-file /dev/null --submit 600 ${BORROWER_ADDR[@]}

TxId "4c7384d54b14df023164375019428df94f32a92ed33be375e947311a0dfc35e5"


### Mint the Djed

The contract will use fake Djed for payments.

In [23]:
DJED_NAME=DjedUSD

In [24]:
DJED_POLICY=$(
marlowe-cli util mint --required-signer $ADMIN_SKEY \
                      --change-address $ADMIN_ADDR  \
                      --expires 1000000011          \
                      --count $TOTAL_DJED           \
                      --out-file /dev/null          \
                      --submit 600                  \
                      $DJED_NAME                    \
| sed -e 's/^PolicyID "\(.*\)"$/\1/'                \
)
echo 'PolicyID "'$DJED_POLICY'"'

PolicyID "76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf"


In [25]:
DJED=$DJED_POLICY.$DJED_NAME
echo $DJED

76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf.DjedUSD


In `cardano-cli` the token name is printed in hexadecimal, so we have . . .

In [26]:
echo $DJED_POLICY.$(echo -n $DJED_NAME | basenc --base16 | tr '[:upper:]' '[:lower:]')

76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf.446a6564555344


### Distribute the Djed.

Send Djed to lenders and borrowers.

In [27]:
TXOUT=()
for j in `seq 1 $N_LENDERS`
do
  TXOUT+=("--tx-out" "${LENDER_ADDR[$j]}+$MIN_ADA+$MAXIMUM_INVESTMENT\ $DJED")
done
for j in `seq 1 $N_BORROWERS`
do
  TXOUT+=("--tx-out" "${BORROWER_ADDR[$j]}+$MIN_ADA+$BORROWER_DJED\ $DJED")
done

In [28]:
echo marlowe-cli transaction simple --tx-in "$(find_ada $ADMIN_ADDR)"         \
                                    --tx-in "$(find_token $ADMIN_ADDR $DJED)" \
                                    ${TXOUT[@]}                               \
                                    --change-address $ADMIN_ADDR              \
                                    --required-signer $ADMIN_SKEY             \
                                    --out-file /dev/null                      \
                                    --submit 600                              \
| bash

TxId "a4876508d5f197c2fbf678989e51f70b35650b6cf5a70f6bff4908c40d599a91"


### Mint the Liquidity Tokens

The contract will use liquidity tokens for bookkeeping.

In [29]:
LIQUIDITY_NAME=Liquidity

In [30]:
LIQUIDITY_POLICY=$(
marlowe-cli util mint --required-signer $ADMIN_SKEY \
                      --change-address $ADMIN_ADDR  \
                      --expires 1000000012          \
                      --count $POOL_SIZE            \
                      --out-file /dev/null          \
                      --submit 600                  \
                      $LIQUIDITY_NAME               \
| sed -e 's/^PolicyID "\(.*\)"$/\1/'                \
)
echo 'PolicyID "'$LIQUIDITY_POLICY'"'

PolicyID "5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd"


In [31]:
LIQUIDITY=$LIQUIDITY_POLICY.$LIQUIDITY_NAME
echo $LIQUIDITY

5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.Liquidity


In `cardano-cli` the token name is printed in hexadecimal, so we have . . .

In [32]:
echo $LIQUIDITY_POLICY.$(echo -n $LIQUIDITY_NAME | basenc --base16 | tr '[:upper:]' '[:lower:]')

5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.4c6971756964697479


### Mint the Role Tokens

There is one role token for the administrator.

In [33]:
ADMIN_ROLE=A

Lender and borrower tokens will be distribute on a just-in-time basis for making deposits and (separately) for making withdrawals. Note that ordering of these is arbitrary and does not necessarily correspond to the lender and borrower addresses.

In [34]:
for j in `seq 1 $N_LENDERS`
do
  LENDER_DEPOSIT_ROLE[$j]=LD$j
  LENDER_PAYMENT_ROLE[$j]=LP$j
done

In [35]:
for j in `seq 1 $N_BORROWERS`
do
  BORROWER_DEPOSIT_ROLE[$j]=BD$j
  BORROWER_PAYMENT_ROLE[$j]=BP$j
done

Mint the role tokens.

In [36]:
ROLES_POLICY=$(
marlowe-cli util mint --required-signer $ADMIN_SKEY \
                      --change-address $ADMIN_ADDR  \
                      --expires 1000000013          \
                      --out-file /dev/null          \
                      --submit 600                  \
                      $ADMIN_ROLE                   \
                      ${LENDER_DEPOSIT_ROLE[@]}     \
                      ${LENDER_PAYMENT_ROLE[@]}     \
                      ${BORROWER_DEPOSIT_ROLE[@]}   \
                      ${BORROWER_PAYMENT_ROLE[@]}   \
| sed -e 's/^PolicyID "\(.*\)"$/\1/'                \
)
echo 'PolicyID "'$ROLES_POLICY'"'

PolicyID "9cb30bdd840cc8b31806c20eae6ed14614d32f10379547d605064e78"


View the role tokens.

In [37]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ADMIN_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
c8dd715210871791f53e2a85902cba7e523a2113044c5e5e7e992e1b1a7b6cbe     0        17382984566 lovelace + TxOutDatumNone
c8dd715210871791f53e2a85902cba7e523a2113044c5e5e7e992e1b1a7b6cbe     1        10000000 lovelace + 150000000 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.4c6971756964697479 + TxOutDatumNone
c8dd715210871791f53e2a85902cba7e523a2113044c5e5e7e992e1b1a7b6cbe     2        10000000 lovelace + 1 9cb30bdd840cc8b31806c20eae6ed14614d32f10379547d605064e78.41 + TxOutDatumNone
c8dd715210871791f53e2a85902cba7e523a2113044c5e5e7e992e1b1a7b6cbe     3        10000000 lovelace + 1 9cb30bdd840cc8b31806c20eae6ed14614d32f10379547d605064e78.424431 + TxOutDatumNone
c8dd715210871791f53e2a85902cba7e523a2113044c5e5e7e992e1b1a7b6cbe     4        10000000 lovelace + 1 9cb30bdd840cc8b31806c20eae6ed14614d32f10379547d605064

## Generate the Contract

Run a Haskell program that generates the contract.

In [38]:
runhaskell << EOI

{-# LANGUAGE OverloadedStrings  #-}


module CollectiveLoan where


import Control.Monad.Writer (Writer, runWriter, tell)
import Data.Aeson ((.=), encodeFile,object)
import Data.Foldable (foldrM)
import Data.String (fromString)
import Language.Marlowe.CLI.Merkle (deepMerkleize)
import Language.Marlowe.CLI.Types (Continuations)
import Language.Marlowe.Core.V1.Semantics.Types
import Plutus.V1.Ledger.Api (POSIXTime(..))


main :: IO ()
main =
  let
    (contract, continuations) =
      runWriter
        $ makeContract
          $POOL_SIZE $SETUP_DEADLINE
          $N_LENDERS   $MINIMUM_INVESTMENT $MAXIMUM_INVESTMENT $INVESTMENT_DEADLINE     $INVESTMENT_REPAYMENT_DEADLINE
          $N_BORROWERS $MINIMUM_LOAN       $MAXIMUM_LOAN       $LOAN_REPAYMENT_DEADLINE $LOAN_DEADLINE
  in
    encodeFile "collective-loan.merkleization"
      $ object
        [
          "contract"      .= contract
        , "continuations" .= continuations
        ]


djed :: Token
djed = Token "$DJED_POLICY" "$DJED_NAME"

liquidity :: Token
liquidity = Token "$LIQUIDITY_POLICY" "$LIQUIDITY_NAME"

equity :: ValueId
equity = "Equity"

liability :: ValueId
liability = "Liability"

administrator :: Party
administrator = Role "$ADMIN_ROLE"


merkleizeTimeout :: Contract -> Writer Continuations Contract
merkleizeTimeout continuation =
  deepMerkleize
    $ When
      [
        Case (Notify TrueObs )
        continuation
      ]
      (POSIXTime $ULTIMATE_DEADLINE)
      Close


makeContract :: Integer -> Timeout
             -> Int -> Integer -> Integer -> Timeout -> Timeout
             -> Int -> Integer -> Integer -> Timeout -> Timeout
             -> Writer Continuations Contract
makeContract poolSize setupDeadline
         nLenders minimumInvestment maximumInvestment lenderDepositDeadline lenderWithdrawalDeadline
         nBorrowers   minimumLoan       maximumLoan       borrowerDepositDeadline   borrowerWithdrawalDeadline =
  let
    lenders = [1..nLenders]
    borrowers = [1..nBorrowers]
    z c = flip (foldrM (lenderWithdraws maximumInvestment lenderWithdrawalDeadline)) lenders c
  in
    setup poolSize setupDeadline
      =<< flip (foldrM $ lenderDeposits minimumInvestment maximumInvestment lenderDepositDeadline) lenders
      =<< pure . computeLiability
      =<< flip (foldrM $ borrowerWithdraws minimumLoan maximumLoan borrowerWithdrawalDeadline) borrowers
      =<< flip (foldrM $ borrowerDeposits maximumLoan borrowerDepositDeadline) borrowers
      =<< pure . computeEquity
      =<< flip (foldrM $ lenderWithdraws maximumInvestment lenderWithdrawalDeadline) lenders
      =<< merkleizeTimeout Close


setup :: Integer -> Timeout -> Contract -> Writer Continuations Contract
setup poolSize deadline continuation =
  deepMerkleize
    $ When
      [
        Case (Deposit administrator administrator liquidity $ Constant poolSize)
          continuation
      ]
      deadline
      Close


computeLiability :: Contract -> Contract
computeLiability =
  Let "Liability"
    $ AvailableMoney administrator djed


computeEquity :: Contract -> Contract
computeEquity =
  Let "Equity"
    $ AvailableMoney administrator djed


borrowerDeposits :: Integer -> Timeout -> Int -> Contract -> Writer Continuations Contract
borrowerDeposits maximumLoan deadline i continuation =
  let
    name = "Borrower Deposit"
    choice = ChoiceId (fromString name) administrator
    amount = ChoiceValue choice
    borrower = Role . fromString $ "BD" <> show i
  in
    deepMerkleize
      . When
        [
          Case (Choice choice [Bound 1 $ 2 * maximumLoan])
            $ When
               [
                 Case (Deposit administrator borrower djed amount)
                   continuation
               ]
               deadline
               continuation
        ]
        deadline
        =<< merkleizeTimeout continuation


borrowerWithdraws :: Integer -> Integer -> Timeout -> Int -> Contract -> Writer Continuations Contract
borrowerWithdraws minimumLoan maximumLoan deadline i continuation =
  let
    name = "Borrower Withdrawal"
    choice = ChoiceId (fromString name) administrator
    amount = ChoiceValue choice
    borrower = Role . fromString $ "BP" <> show i
  in
    deepMerkleize
      . When
        [
          Case (Choice choice [Bound minimumLoan maximumLoan])
            $ Pay administrator (Party borrower) djed amount
              continuation
        ]
        deadline
        =<< merkleizeTimeout continuation


lenderDeposits :: Integer -> Integer -> Timeout -> Int -> Contract -> Writer Continuations Contract
lenderDeposits minimumInvestment maximumInvestment deadline i continuation =
  let
    name = "Lender Deposit"
    choice = ChoiceId (fromString name) administrator
    amount = ChoiceValue choice
    lender = Role . fromString $ "LD" <> show i
  in
    deepMerkleize
      . When
      [
        Case (Choice choice [Bound minimumInvestment maximumInvestment])
          $ When
              [
                Case (Deposit administrator lender djed amount)
                  $ Pay administrator (Party lender) liquidity amount
                    continuation
              ]
              deadline
              continuation
      ]
      deadline
      =<< merkleizeTimeout continuation


lenderWithdraws :: Integer -> Timeout -> Int -> Contract -> Writer Continuations Contract
lenderWithdraws maximumInvestment deadline i continuation =
  let
    name = "Lender Withdrawal"
    choice = ChoiceId (fromString name) administrator
    amount = ChoiceValue choice
    lender = Role . fromString $ "LP" <> show i
  in
    deepMerkleize
      . When
        [
          Case (Choice choice [Bound 1 $ 2 * maximumInvestment])
            $ When
                [
                  Case (Deposit administrator lender liquidity amount)
                    $ Pay administrator (Party lender) djed
                      (
                        DivValue
                          (MulValue amount $ UseValue equity)
                          (UseValue liability)
                      )
                      continuation
                ]
                deadline
                continuation
        ]
        deadline
        =<< merkleizeTimeout continuation
    
EOI

How large is the contract?

In [39]:
ls -lh collective-loan.merkleization

-rw-rw-r-- 1 bbush bbush-upg 333K Aug 14 11:31 collective-loan.merkleization


How many `Case` statements does it contain?

In [40]:
sed -e 's/case/&\n/g' collective-loan.merkleization | grep case | wc -l

1050


How many merkleizations does it contain?

In [41]:
jq '.continuations | length' collective-loan.merkleization

461


Separate the contract from the merkleizations.

In [42]:
jq .contract collective-loan.merkleization > collective-loan-1.contract

## Set the initial state

The Seller will deposit 2 Ada when the contract is created.

In [43]:
yaml2json << EOI > collective-loan-1.state
accounts:
- - - role_token: $ADMIN_ROLE
    - currency_symbol: ''
      token_name: ''
  - $MIN_ADA
boundValues: []
choices: []
minTime: $NOW
EOI

In [44]:
cat collective-loan-1.state

{"accounts":[[[{"role_token":"A"},{"currency_symbol":"","token_name":""}],2000000]],"boundValues":[],"choices":[],"minTime":1660498029000}


## Simulate and Execute the Contract

### 1. Initialize the contract information

Create a `.marlowe` file that contains all of the information needed to start the contract. Note that we merkleization the contract because of its large size.

In [45]:
marlowe-cli run initialize --roles-currency $ROLES_POLICY             \
                           --contract-file collective-loan-1.contract \
                           --state-file    collective-loan-1.state    \
                           --out-file      collective-loan-1.partial  \
                           --print-stats


Validator size: 12611
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 24562825, exBudgetMemory = ExMemory 82600}


Manually merge the continuations into the `.marlowe` file.

In [46]:
jq -s '.[0] * .[1]' collective-loan-1.partial collective-loan.merkleization > collective-loan-1.marlowe

The minimum Ada deposit has been recorded.

In [47]:
jq .state collective-loan-1.marlowe | json2yaml

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
boundValues: []
choices: []
minTime: 1660498029000


Submit the transaction.

In [48]:
TX_1=$(
marlowe-cli run execute --tx-in "$(find_ada $ADMIN_ADDR)"            \
                        --marlowe-out-file collective-loan-1.marlowe \
                        --change-address $ADMIN_ADDR                 \
                        --required-signer $ADMIN_SKEY                \
                        --out-file /dev/null                         \
                        --submit 600                                 \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                     \
)
echo 'TxId "'$TX_1'"'

TxId "cc66c5bcf405f3c76f9bfc7fc1f8d7a154f772cc8bc335d835453945bd947bf5"


View the UTxO at the script address.

In [49]:
SCRIPT_ADDR=$(jq -r .marloweValidator.address collective-loan-1.marlowe)
echo $SCRIPT_ADDR

addr_test1wphttveup3kdrp7ch4e4xjr8jxstwwx20fg8s4epqayylfqga9h6p


In [50]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
cc66c5bcf405f3c76f9bfc7fc1f8d7a154f772cc8bc335d835453945bd947bf5     1        2000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "ab031567814f9f08b13b8c6c9f0973979d030e4c78d40c7db93ad0d00b88a020"


### 2. The administrator deposits the liquidity tokens

Prepare the transaction.

In [51]:
marlowe-cli run prepare --deposit-account Role=$ADMIN_ROLE                   \
                        --deposit-party Role=$ADMIN_ROLE                     \
                        --deposit-token $LIQUIDITY                           \
                        --deposit-amount $POOL_SIZE                          \
                        --invalid-before $NOW                                \
                        --invalid-hereafter $((SETUP_DEADLINE - 1 * MINUTE)) \
                        --marlowe-file collective-loan-1.marlowe             \
                        --out-file     collective-loan-2.marlowe             \
                        --print-stats


Datum size: 262


Now the contract contains both the initial Ada and the liquidity tokens.

In [52]:
jq .state collective-loan-2.marlowe | json2yaml

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: A
    - currency_symbol: 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd
      token_name: Liquidity
  - 150000000
boundValues: []
choices: []
minTime: 1660498029000


Submit the transaction.

In [53]:
TX_2=$(
marlowe-cli run execute --marlowe-in-file collective-loan-1.marlowe           \
                        --tx-in-marlowe "$TX_1#1"                             \
                        --tx-in-collateral "$TX_1#0"                          \
                        --tx-in "$TX_1#0"                                     \
                        --tx-in "$(find_token $ADMIN_ADDR $LIQUIDITY)"        \
                        --tx-in "$(find_role_token $ADMIN_ADDR $ADMIN_ROLE)"  \
                        --tx-out "$(make_role_txout $ADMIN_ADDR $ADMIN_ROLE)" \
                        --marlowe-out-file collective-loan-2.marlowe          \
                        --change-address $ADMIN_ADDR                          \
                        --required-signer $ADMIN_SKEY                         \
                        --out-file /dev/null                                  \
                        --submit 600                                          \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                              \
)
echo 'TxId "'$TX_2'"'

TxId "ed444ca28664dba7c473f2c10015ff01319a7adda12853dd73f09d5f36079c5d"


View the UTxO at the script address.

In [54]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
ed444ca28664dba7c473f2c10015ff01319a7adda12853dd73f09d5f36079c5d     1        2000000 lovelace + 150000000 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.4c6971756964697479 + TxOutDatumHash ScriptDataInAlonzoEra "ae11c966dbbc5ca2af85747a7c7b69baa7439bef6ca6cad63dc27e959040711f"


### 3. The lenders deposit their investment funds

Lenders deposit in random order.

In [55]:
INVESTMENT_ORDER=(`seq 1 $N_LENDERS | sort -R`)
echo ${INVESTMENT_ORDER[@]}

30 23 1 17 24 19 13 8 5 14 28 7 6 27 22 9 29 4 11 18 21 3 26 20 16 10 2 12 15 25


Lenders deposit random amounts.

In [56]:
for j in `seq 1 $N_LENDERS`
do
  INVESTMENT_AMOUNT[$j]=$(((MAXIMUM_INVESTMENT - MINIMUM_INVESTMENT) * RANDOM / 32767 + MINIMUM_INVESTMENT))
done
echo ${INVESTMENT_AMOUNT[@]}

3555528 4785073 4626316 1020081 3644520 1795876 3653859 4724234 3257240 3258201 1418759 4069841 3043412 4428006 1135441 2922421 2115863 2493804 3020477 4076982 3875926 2722052 4464262 2174642 676198 4707754 2148960 2069444 4797845 1766899


Make the deposits.

In [57]:
TX_OLD=$TX_2
k=2
for i in `seq 1 $N_LENDERS`
do
  k1=$((k+1))
  k2=$((k+2))
  j=${INVESTMENT_ORDER[$((i-1))]}
  echo
  echo "--- Lender deposit #$i ---"
  ROLE=${LENDER_DEPOSIT_ROLE[$i]}
  ADDR=${LENDER_ADDR[$j]}
  SKEY=${LENDER_SKEY[$j]}
  AMOUNT=${INVESTMENT_AMOUNT[$j]}
  DEADLINE=$((INVESTMENT_DEADLINE - 1 * MINUTE))
  echo "Lender $j requests to deposit $AMOUNT Djed."
  marlowe-cli run prepare --choice-party Role=$ADMIN_ROLE            \
                          --choice-name "Lender Deposit"             \
                          --choice-number $AMOUNT                    \
                          --invalid-before $NOW                      \
                          --invalid-hereafter $DEADLINE              \
                          --marlowe-file collective-loan-$k.marlowe  \
                          --out-file     collective-loan-$k1.marlowe
  echo "Administrator approves deposit and sends role token to lender."
  TX_ADA=$(find_ada $ADMIN_ADDR)
  TX_NEW=$(
  marlowe-cli run execute --marlowe-in-file collective-loan-$k.marlowe          \
                          --tx-in-marlowe "$TX_OLD#1"                           \
                          --tx-in-collateral $TX_ADA                            \
                          --tx-in $TX_ADA                                       \
                          --tx-in "$(find_role_token $ADMIN_ADDR $ADMIN_ROLE)"  \
                          --tx-in "$(find_role_token $ADMIN_ADDR $ROLE)"        \
                          --tx-out "$(make_role_txout $ADMIN_ADDR $ADMIN_ROLE)" \
                          --tx-out "$(make_role_txout $ADDR $ROLE)"             \
                          --marlowe-out-file collective-loan-$k1.marlowe        \
                          --change-address $ADMIN_ADDR                          \
                          --required-signer $ADMIN_SKEY                         \
                          --out-file /dev/null                                  \
                          --submit 600                                          \
  | sed -e 's/^TxId "\(.*\)"$/\1/'                                              \
  )
  echo 'TxId "'$TX_NEW'"'
  TX_OLD=$TX_NEW
  echo "Lender deposits Djed and receives liquidity tokens using their newly-received role token."
  marlowe-cli run prepare --deposit-account Role=$ADMIN_ROLE         \
                          --deposit-party Role=$ROLE                 \
                          --deposit-amount $AMOUNT                   \
                          --deposit-token $DJED                      \
                          --invalid-before $NOW                      \
                          --invalid-hereafter $DEADLINE              \
                          --marlowe-file collective-loan-$k1.marlowe \
                          --out-file     collective-loan-$k2.marlowe 
  TX_ADA=$(find_ada $ADDR)
  CHANGE=$((MAXIMUM_INVESTMENT - AMOUNT))
  TX_NEW=$(
  marlowe-cli run execute --marlowe-in-file collective-loan-$k1.marlowe      \
                          --tx-in-marlowe "$TX_OLD#1"                        \
                          --tx-in-collateral $TX_ADA                         \
                          --tx-in $TX_ADA                                    \
                          --tx-in "$(find_token $ADDR $DJED)"                \
                          --tx-in "$(find_role_token $ADDR $ROLE)"           \
                          --tx-out "$(make_role_txout $ADDR $ROLE)"          \
                          --tx-out "$(make_token_txout $ADDR $DJED $CHANGE)" \
                          --marlowe-out-file collective-loan-$k2.marlowe     \
                          --change-address $ADDR                             \
                          --required-signer $SKEY                            \
                          --out-file /dev/null                               \
                          --submit 600                                       \
  | sed -e 's/^TxId "\(.*\)"$/\1/'                                           \
  )
  echo 'TxId "'$TX_NEW'"'
  TX_OLD=$TX_NEW
  k=$k2
done


--- Lender deposit #1 ---
Lender 30 requests to deposit 1766899 Djed.
Administrator approves deposit and sends role token to lender.
TxId "5dd01d1aa5d46571e922db72df8472bc0b63b024c8ff3147b9523cb5c7d95398"
Lender deposits Djed and receives liquidity tokens using their newly-received role token.
Payment 1
  Acccount: "A"
  Payee: Party "LD1"
  Ada: 0.000000
  5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd."Liquidity": 1766899
TxId "61cb268abe7fb9182cde676052b77b6fe10a2fbe56dd54d48294324166dd4f76"

--- Lender deposit #2 ---
Lender 23 requests to deposit 4464262 Djed.
Administrator approves deposit and sends role token to lender.
TxId "cebe5688b8d28923d1a7b3c6ffaf7e7d456add9178da14f2482a49b829ade9e7"
Lender deposits Djed and receives liquidity tokens using their newly-received role token.
Payment 1
  Acccount: "A"
  Payee: Party "LD2"
  Ada: 0.000000
  5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd."Liquidity": 4464262
TxId "513f9980606d579577dddaf2513e6c1cffa4cb25d0a1

View the state of the contract.

In [58]:
jq .state collective-loan-$k.marlowe | json2yaml

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: A
    - currency_symbol: 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd
      token_name: Liquidity
  - 57550084
- - - role_token: A
    - currency_symbol: 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf
      token_name: DjedUSD
  - 92449916
boundValues:
- - Liability
  - 92449916
choices:
- - choice_name: Lender Deposit
    choice_owner:
      role_token: A
  - 676198
minTime: 1660498029000


View the UTxO at the script address.

In [59]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
e136662c1329cbd8988ba3fc049394f77aa6375281acb3f8afe51d7f78b01e5d     1        2000000 lovelace + 57550084 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.4c6971756964697479 + 92449916 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf.446a6564555344 + TxOutDatumHash ScriptDataInAlonzoEra "a47490c2f47356dacb04ce4cd22eefe12cbc1e29bf4cdc681ab30e8501673fb1"


### 4. The lenders withdraw their liquidity tokens

The role-payout validator address contains the liquidity tokens that were paid to the lenders.

In [60]:
ROLES_ADDR=$(jq -r .rolesValidator.address collective-loan-1.marlowe)
echo $ROLES_ADDR

addr_test1wq0fq8ghuemq7cjk4cd3fys20g6ew4ntpv7tpw4fclat8nqs93zsf


View these payouts.

In [61]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ROLES_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
03cb2f63e77f06bed956fe401d644593227b29e0038538d93725eaf1f51eeb0f     2        1724100 lovelace + 4069841 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.4c6971756964697479 + TxOutDatumHash ScriptDataInAlonzoEra "a8660a918e0595371c4a4a555526c269a38b9f021312de8a5e465c02335c1642"
0dfc40206c1da61a5c0794e94b1a3ab596b66c5f38df1477039d1ddf0e6b7449     2        1724100 lovelace + 2493804 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.4c6971756964697479 + TxOutDatumHash ScriptDataInAlonzoEra "e6511d59e75c7bd5fd7044c2f8f5dd10ffce3d99ddafc1f0ef47638455cfb22f"
2579e17f8237562e6ea03dce900a616411115d5f95295c88937a89427d008db7     2        1724100 lovelace + 3875926 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.4c6971756964697479 + TxOutDatumHash ScriptDataInAlonzoEra "9256a756fd306bcd5afa8bc8dcf5260c7

Withdraw these payouts to the lenders' addresses.

In [62]:
for i in `seq 1 $N_LENDERS`
do
  j=${INVESTMENT_ORDER[$((i-1))]}
  ROLE=${LENDER_DEPOSIT_ROLE[$i]}
  ADDR=${LENDER_ADDR[$j]}
  SKEY=${LENDER_SKEY[$j]}
  TX_ADA=$(find_ada $ADDR)
  marlowe-cli run withdraw --marlowe-file collective-loan-$((2*j+2)).marlowe \
                           --role-name $ROLE                                 \
                           --tx-in-collateral $TX_ADA                        \
                           --tx-in $TX_ADA                                   \
                           --tx-in $(find_role_token $ADDR $ROLE)            \
                           --tx-out "$(make_role_txout $ADDR $ROLE)"         \
                           --change-address $ADDR                            \
                           --required-signer $SKEY                           \
                           --out-file /dev/null                              \
                           --submit 600
done

TxId "579014ac8b3d3b4d219c2a5251c7728f710bb89260a1927776c253325259a65e"
TxId "b1fd9a256dcee84a7e56abdd8efe2b68c19698e937befd40d1bac32804c9065a"
TxId "a39d289baf98d071ea77cb6632bcbca92f1154a7b113aca752df79673014fe1e"
TxId "f4d3a27809c7012f82a9121ab6f161086f5b45007eb94f1e7848a475cd60def8"
TxId "388bce6fbc504b0c6f738add83c3d256ca36a309e4dbbbf9f778a5bb368e7072"
TxId "07f0e7c2a1cdc46d99ba6d3bebe352a9b36363bc5f6b59900610f7c1c5754068"
TxId "f428bdbfb6045f4c5c326c22f13c8975debced7c593f6814a8e33ffd7c86931c"
TxId "c3e7d927232840b3b6ad669074ba8328173fc945b3a73a412de2d8339c5507eb"
TxId "7f2357fcd7cc41ab39b8a7f73703bd627034b03930608fc310a50caef34bb149"
TxId "53efef7ed7b6c53656795bbf913c34e6bf92e412d2b30108c3c9cd0cd4f85518"
TxId "3d01ab3ed4ebbe62e983a1d49cf9b0f863b678fbd6a80ba5cfaaa07250c2d404"
TxId "6447d873c2d763bf10f1863424797e4c1f6c0cb3d22bd4c7483def7b5305bba8"
TxId "d30ba8539a9c324b601658e3323403eaefe7e532328c860feb0c27b4e7352676"
TxId "4d7657b71394473fae6311198dd1152cbd5930e289e141efe3d5800ff6

There are no UTxOs at the payout address.

In [63]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ROLES_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


View the UTxOs at the lenders' addresses to see that they have their role tokens and liquidity tokens.

In [64]:
for j in `seq 1 $N_LENDERS`
do
  echo
  echo "Lender $j = ${LENDER_ADDR[$j]}"
  cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address ${LENDER_ADDR[$j]}
done


Lender 1 = addr_test1qzenwj7elwatk3mmc368mwnv067fqnuk3x7u4nqw5dezg8wr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqf6k36p
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
a39d289baf98d071ea77cb6632bcbca92f1154a7b113aca752df79673014fe1e     0        19375281 lovelace + TxOutDatumNone
a39d289baf98d071ea77cb6632bcbca92f1154a7b113aca752df79673014fe1e     1        1724100 lovelace + 3555528 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.4c6971756964697479 + TxOutDatumNone
a39d289baf98d071ea77cb6632bcbca92f1154a7b113aca752df79673014fe1e     2        2000000 lovelace + 1 9cb30bdd840cc8b31806c20eae6ed14614d32f10379547d605064e78.4c4433 + TxOutDatumNone
a6741af95cb0b18f7b8063b4bb85079e35430df91c0f17e952700935d4d9dc75     4        2000000 lovelace + 1444472 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf.446a6564555344 + TxOutDatumNone
a6741af95cb0b18f7b8063

### 5. The borrowers withdraw their loan funds

Borrowers withdraw in random order.

In [65]:
BORROWER_ORDER=(`seq 1 $N_BORROWERS | sort -R`)
echo ${BORROWER_ORDER[@]}

49 66 7 64 10 58 53 54 31 23 33 40 2 29 47 15 14 59 11 43 4 56 37 55 57 44 26 48 50 28 41 18 32 65 62 34 13 69 22 17 46 52 67 12 24 70 68 35 3 63 51 27 45 60 21 16 6 5 36 25 61 42 8 30 1 39 20 9 19 38


Borrowers deposit random amounts.

In [66]:
for j in `seq 1 $N_BORROWERS`
do
  LOAN_AMOUNT[$j]=$(((MAXIMUM_LOAN - MINIMUM_LOAN) * RANDOM / 32767 + MINIMUM_LOAN))
done
echo ${LOAN_AMOUNT[@]}

588109 561577 791390 579705 299353 843412 570311 985140 487307 464427 457203 213437 842725 208355 201297 380516 307895 859700 299160 388729 230439 180422 530704 174736 552815 317865 318991 361043 688885 996951 735358 863435 643372 539274 761369 871373 252659 893539 201434 300479 109887 207971 631040 288613 294958 733820 243733 225193 237113 463191 568883 246232 483516 887881 192809 379857 375600 122055 910788 311410 518674 343519 443086 675975 652546 835694 165617 471431 333301 673943


Make the withdrawals.

In [67]:
for i in `seq 1 $N_BORROWERS`
do
  k1=$((k+1))
  j=${BORROWER_ORDER[$((i-1))]}
  echo
  echo "--- Borrower withdrawal #$i ---"
  ROLE=${BORROWER_PAYMENT_ROLE[$i]}
  ADDR=${BORROWER_ADDR[$j]}
  SKEY=${BORROWER_SKEY[$j]}
  AMOUNT=${LOAN_AMOUNT[$j]}
  echo "Borrower $j requests loan of $AMOUNT Djed."
  echo "Administrator checks credit."
  echo "Administator approves loan and sends role token to borrower."
  DEADLINE=$((LOAN_DEADLINE - 1 * MINUTE))
  marlowe-cli run prepare --choice-party Role=$ADMIN_ROLE            \
                          --choice-name "Borrower Withdrawal"        \
                          --choice-number $AMOUNT                    \
                          --invalid-before $NOW                      \
                          --invalid-hereafter $DEADLINE              \
                          --marlowe-file collective-loan-$k.marlowe  \
                          --out-file     collective-loan-$k1.marlowe
  TX_ADA=$(find_ada $ADMIN_ADDR)
  TX_NEW=$(
  marlowe-cli run execute --marlowe-in-file collective-loan-$k.marlowe          \
                          --tx-in-marlowe "$TX_OLD#1"                           \
                          --tx-in-collateral $TX_ADA                            \
                          --tx-in $TX_ADA                                       \
                          --tx-in "$(find_role_token $ADMIN_ADDR $ADMIN_ROLE)"  \
                          --tx-in "$(find_role_token $ADMIN_ADDR $ROLE)"        \
                          --tx-out "$(make_role_txout $ADMIN_ADDR $ADMIN_ROLE)" \
                          --tx-out "$(make_role_txout $ADDR $ROLE)"             \
                          --marlowe-out-file collective-loan-$k1.marlowe        \
                          --change-address $ADMIN_ADDR                          \
                          --required-signer $ADMIN_SKEY                         \
                          --out-file /dev/null                                  \
                          --submit 600                                          \
  | sed -e 's/^TxId "\(.*\)"$/\1/'                                              \
  )
  echo 'TxId "'$TX_NEW'"'
  TX_OLD=$TX_NEW
  echo "Borrower withdraws the Djed from the payout address using their newly-received role token."
  TX_ADA=$(find_ada $ADDR)
  marlowe-cli run withdraw --marlowe-file collective-loan-$k1.marlowe \
                           --role-name $ROLE                          \
                           --tx-in-collateral $TX_ADA                 \
                           --tx-in $TX_ADA                            \
                           --tx-in $(find_role_token $ADDR $ROLE)     \
                           --tx-out "$(make_role_txout $ADDR $ROLE)"  \
                           --change-address $ADDR                     \
                           --required-signer $SKEY                    \
                           --out-file /dev/null                       \
                           --submit 600
  k=$k1
done


--- Borrower withdrawal #1 ---
Borrower 49 requests loan of 237113 Djed.
Administrator checks credit.
Administator approves loan and sends role token to borrower.
Payment 1
  Acccount: "A"
  Payee: Party "BP1"
  Ada: 0.000000
  76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf."DjedUSD": 237113
TxId "c39e0d9ab5c9a6f392a12b9984c8838644a03820511744488a04f8238cf19eef"
Borrower withdraws the Djed from the payout address using their newly-received role token.
TxId "36c39904c378191444dbf2be1feebdc0a4cefd831ec1b20ec346b2aec038417c"

--- Borrower withdrawal #2 ---
Borrower 66 requests loan of 835694 Djed.
Administrator checks credit.
Administator approves loan and sends role token to borrower.
Payment 1
  Acccount: "A"
  Payee: Party "BP2"
  Ada: 0.000000
  76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf."DjedUSD": 835694
TxId "9220d4768a328c6241fc8eb58b064dd1fcabf1329640424ab2015ba541d511ec"
Borrower withdraws the Djed from the payout address using their newly-received role t

Now the contract accounts show that some Djed remains.

In [68]:
jq .state.accounts collective-loan-$k.marlowe | json2yaml

- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: A
    - currency_symbol: 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd
      token_name: Liquidity
  - 57550084
- - - role_token: A
    - currency_symbol: 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf
      token_name: DjedUSD
  - 58676689


View the UTxO at the script address.

In [69]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
bdf3678bc6b9a80e382410e7d9043b8e5a0cce9b33c36d38140d61fbf92c43d4     1        2000000 lovelace + 57550084 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.4c6971756964697479 + 58676689 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf.446a6564555344 + TxOutDatumHash ScriptDataInAlonzoEra "6e24f5d4632d6210c6f5fe1fe89b598491e668de046fd3bd7c8ade7561199094"


There are no UTxOs at the payout address.

In [70]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ROLES_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


View the UTxOs at the borrowers' addresses to see that they have their role tokens and Djed.

In [71]:
for j in `seq 1 $N_BORROWERS`
do
  echo
  echo "Borrower $j = ${BORROWER_ADDR[$j]}"
  cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address ${BORROWER_ADDR[$j]}
done


Borrower 1 = addr_test1qp7q4e9nlhq08mrugs8pm5uq045e425w72mxhxupq9qqgykr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqjmt3lh
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
8857c790adebcf6fa85cc56eef308304800cca0996c2aa32ab3f1550f52969e9     0        24551679 lovelace + TxOutDatumNone
8857c790adebcf6fa85cc56eef308304800cca0996c2aa32ab3f1550f52969e9     1        1689618 lovelace + 588109 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf.446a6564555344 + TxOutDatumNone
8857c790adebcf6fa85cc56eef308304800cca0996c2aa32ab3f1550f52969e9     2        2000000 lovelace + 1 9cb30bdd840cc8b31806c20eae6ed14614d32f10379547d605064e78.42503635 + TxOutDatumNone
a4876508d5f197c2fbf678989e51f70b35650b6cf5a70f6bff4908c40d599a91    31        2000000 lovelace + 500000 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf.446a6564555344 + TxOutDatumNone

Borrower 2 = addr_test1

### 6. The borrowers pay back their funds with interest

Some time passes, and the borrowers pay back their loans. We randomize the sequence of paybacks and returns.

In [72]:
BORROWER_ORDER=(`seq 1 $N_BORROWERS | sort -R`)
echo ${BORROWER_ORDER[@]}

57 69 37 48 59 10 42 24 67 13 33 26 40 7 53 18 70 66 20 43 3 15 32 28 1 65 27 52 22 46 2 16 41 55 68 12 17 25 50 47 39 19 34 31 35 4 51 36 14 8 38 56 6 63 58 29 5 23 60 62 44 54 49 64 30 45 11 9 61 21


In [73]:
for j in `seq 1 $N_BORROWERS`
do
  INTEREST_PERCENT=$((50 * RANDOM / 32767 - 10))
  RETURN_AMOUNT[$j]=$((${LOAN_AMOUNT[$j]} * (100 + INTEREST_PERCENT) / 100))
done
echo ${LOAN_AMOUNT[@]}
echo ${RETURN_AMOUNT[@]}

588109 561577 791390 579705 299353 843412 570311 985140 487307 464427 457203 213437 842725 208355 201297 380516 307895 859700 299160 388729 230439 180422 530704 174736 552815 317865 318991 361043 688885 996951 735358 863435 643372 539274 761369 871373 252659 893539 201434 300479 109887 207971 631040 288613 294958 733820 243733 225193 237113 463191 568883 246232 483516 887881 192809 379857 375600 122055 910788 311410 518674 343519 443086 675975 652546 835694 165617 471431 333301 673943
629276 505419 830959 660863 305340 1037396 741404 1231425 599387 575889 448058 271064 994415 210438 239543 365295 335605 962864 329076 415940 241960 248982 626230 204441 585983 429117 389169 379095 640663 1046798 882429 975681 701275 496132 700459 949796 290557 1045440 203448 381608 110985 241246 605798 326132 339201 785187 233983 236452 322473 472454 659904 290553 498021 950032 183168 452029 341796 124496 1129377 289611 471993 453445 438655 703014 756953 1002832 168929 471431 463288 673943


Pay back the loans.

In [74]:
for i in `seq 1 $N_BORROWERS`
do
  k1=$((k+1))
  k2=$((k+2))
  j=${BORROWER_ORDER[$((i-1))]}
  echo
  echo "--- Borrower deposit #$i ---"
  ROLE=${BORROWER_DEPOSIT_ROLE[$i]}
  ADDR=${BORROWER_ADDR[$j]}
  SKEY=${BORROWER_SKEY[$j]}
  AMOUNT=${RETURN_AMOUNT[$j]}
  DEADLINE=$((LOAN_REPAYMENT_DEADLINE - 1 * MINUTE))
  echo "Borrower $j requests to deposit $AMOUNT DjedUSD."
  marlowe-cli run prepare --choice-party Role=$ADMIN_ROLE            \
                          --choice-name "Borrower Deposit"           \
                          --choice-number $AMOUNT                    \
                          --invalid-before $NOW                      \
                          --invalid-hereafter $DEADLINE              \
                          --marlowe-file collective-loan-$k.marlowe  \
                          --out-file     collective-loan-$k1.marlowe
  echo "Administrator approves deposit and sends role token to borrower."
  TX_ADA=$(find_ada $ADMIN_ADDR)
  TX_NEW=$(
  marlowe-cli run execute --marlowe-in-file collective-loan-$k.marlowe          \
                          --tx-in-marlowe "$TX_OLD#1"                           \
                          --tx-in-collateral $TX_ADA                            \
                          --tx-in $TX_ADA                                       \
                          --tx-in "$(find_role_token $ADMIN_ADDR $ADMIN_ROLE)"  \
                          --tx-in "$(find_role_token $ADMIN_ADDR $ROLE)"        \
                          --tx-out "$(make_role_txout $ADMIN_ADDR $ADMIN_ROLE)" \
                          --tx-out "$(make_role_txout $ADDR $ROLE)"             \
                          --marlowe-out-file collective-loan-$k1.marlowe        \
                          --change-address $ADMIN_ADDR                          \
                          --required-signer $ADMIN_SKEY                         \
                          --out-file /dev/null                                  \
                          --submit 600                                          \
  | sed -e 's/^TxId "\(.*\)"$/\1/'                                              \
  )
  echo 'TxId "'$TX_NEW'"'
  TX_OLD=$TX_NEW
  echo "Borrower makes deposit using their newly-received role token."
  marlowe-cli run prepare --deposit-account Role=$ADMIN_ROLE         \
                          --deposit-party Role=$ROLE                 \
                          --deposit-amount $AMOUNT                   \
                          --deposit-token $DJED                      \
                          --invalid-before $NOW                      \
                          --invalid-hereafter $DEADLINE              \
                          --marlowe-file collective-loan-$k1.marlowe \
                          --out-file     collective-loan-$k2.marlowe 
  TX_ADA=$(find_ada $ADDR)
  CHANGE=$((BORROWER_DJED + LOAN_AMOUNT[$j] - AMOUNT))
  TX_NEW=$(
  marlowe-cli run execute --marlowe-in-file collective-loan-$k1.marlowe      \
                          --tx-in-marlowe "$TX_OLD#1"                        \
                          --tx-in-collateral $TX_ADA                         \
                          --tx-in $TX_ADA                                    \
                          --tx-in "$(find_token $ADDR $DJED)"                \
                          --tx-in "$(find_token_2 $ADDR $DJED)"              \
                          --tx-in "$(find_role_token $ADDR $ROLE)"           \
                          --tx-out "$(make_role_txout $ADDR $ROLE)"          \
                          --tx-out "$(make_token_txout $ADDR $DJED $CHANGE)" \
                          --marlowe-out-file collective-loan-$k2.marlowe     \
                          --change-address $ADDR                             \
                          --required-signer $SKEY                            \
                          --out-file /dev/null                               \
                          --submit 600                                       \
  | sed -e 's/^TxId "\(.*\)"$/\1/'                                           \
  )
  echo 'TxId "'$TX_NEW'"'
  TX_OLD=$TX_NEW
  k=$k2
done


--- Borrower deposit #1 ---
Borrower 57 requests to deposit 341796 DjedUSD.
Administrator approves deposit and sends role token to borrower.
TxId "162a8645e1f79d722763350c0a411d8bc0ea7bf587ddcc02bf4117b14cb47600"
Borrower makes deposit using their newly-received role token.
TxId "0cfcd99201895b33460f5e3439cb7e0f70bb1a876c197329eab35efb4e1892e8"

--- Borrower deposit #2 ---
Borrower 69 requests to deposit 463288 DjedUSD.
Administrator approves deposit and sends role token to borrower.
TxId "fbcdd1e53b2e707d021723ed97823bf183a3bffcab4920a5cbfebeb7d6d933c5"
Borrower makes deposit using their newly-received role token.
TxId "3828cd9741badc0cce2de4517f912d9e40e4a02ffe35cd3a7a338451bda36850"

--- Borrower deposit #3 ---
Borrower 37 requests to deposit 290557 DjedUSD.
Administrator approves deposit and sends role token to borrower.
TxId "c544f0f05e7ea6399f471e2ce0a16b491e302bb797a2d09f01b0f807971dfc70"
Borrower makes deposit using their newly-received role token.
TxId "8ab0ebc748bd425f693c7e

Now the contract accounts show that the Djed balance has increased.

In [75]:
jq .state.accounts collective-loan-$k.marlowe | json2yaml

- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: A
    - currency_symbol: 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd
      token_name: Liquidity
  - 57550084
- - - role_token: A
    - currency_symbol: 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf
      token_name: DjedUSD
  - 95982989


The contract has an excess of assets over liabilities.

In [76]:
jq .state.boundValues collective-loan-$k.marlowe | json2yaml

- - Liability
  - 92449916
- - Equity
  - 95982989


View the UTxO at the script address.

In [77]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
0dfb062d5f9dd42e212e98e2b402e383174a403f5639f334314a5e9326ba931f     1        2000000 lovelace + 57550084 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.4c6971756964697479 + 95982989 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf.446a6564555344 + TxOutDatumHash ScriptDataInAlonzoEra "86ba6e30538c4c02df0935bd229632db8478700ecaa2eab096c944e28429a20e"


There are no UTxOs at the payout address.

In [78]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ROLES_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


Defragment and then view the UTxOs at the borrowers' addresses to see that they have their role tokens and Djed.

In [79]:
for j in `seq 1 $N_BORROWERS`
do
  echo
  echo "Borrower $j = ${BORROWER_ADDR[$j]}"
  marlowe-cli util clean --required-signer ${BORROWER_SKEY[$j]} --change-address ${BORROWER_ADDR[$j]} --out-file /dev/null --submit 600
  cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address ${BORROWER_ADDR[$j]}
done


Borrower 1 = addr_test1qp7q4e9nlhq08mrugs8pm5uq045e425w72mxhxupq9qqgykr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqjmt3lh
TxId "cb983bd4d0e41b1c83cf606cbdce8bb80c0d18348749db969b25b62585dd6a48"
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
cb983bd4d0e41b1c83cf606cbdce8bb80c0d18348749db969b25b62585dd6a48     0        24245700 lovelace + TxOutDatumNone
cb983bd4d0e41b1c83cf606cbdce8bb80c0d18348749db969b25b62585dd6a48     1        2000000 lovelace + 458833 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf.446a6564555344 + TxOutDatumNone
cb983bd4d0e41b1c83cf606cbdce8bb80c0d18348749db969b25b62585dd6a48     2        2000000 lovelace + 1 9cb30bdd840cc8b31806c20eae6ed14614d32f10379547d605064e78.42443235 + TxOutDatumNone
cb983bd4d0e41b1c83cf606cbdce8bb80c0d18348749db969b25b62585dd6a48     3        2000000 lovelace + 1 9cb30bdd840cc8b31806c20eae6ed14614d32f10379547

### 7. The lenders redeem their liquidity tokens

Lenders redeem in random order.

In [80]:
INVESTMENT_ORDER=(`seq 1 $N_LENDERS | sort -R`)
echo ${INVESTMENT_ORDER[@]}

18 20 26 8 19 9 29 21 24 3 6 27 10 2 22 1 23 14 7 15 30 25 12 5 17 13 28 4 11 16


Redeem the proceeds of the investments.

In [81]:
for i in `seq 1 $N_LENDERS`
do
  k1=$((k+1))
  k2=$((k+2))
  j=${INVESTMENT_ORDER[$((i-1))]}
  echo
  echo "--- Lender redemption #$i ---"
  ROLE=${LENDER_PAYMENT_ROLE[$i]}
  ADDR=${LENDER_ADDR[$j]}
  SKEY=${LENDER_SKEY[$j]}
  AMOUNT=${INVESTMENT_AMOUNT[$j]}
  DEADLINE=$((INVESTMENT_REPAYMENT_DEADLINE - 1 * MINUTE))
  echo "Lender $j requests to deposit $AMOUNT liquidity tokens."
  marlowe-cli run prepare --choice-party Role=$ADMIN_ROLE            \
                          --choice-name "Lender Withdrawal"          \
                          --choice-number $AMOUNT                    \
                          --invalid-before $NOW                      \
                          --invalid-hereafter $DEADLINE              \
                          --marlowe-file collective-loan-$k.marlowe  \
                          --out-file     collective-loan-$k1.marlowe
  echo "Administrator approves deposit and sends role token to lender."
  TX_ADA=$(find_ada $ADMIN_ADDR)
  TX_NEW=$(
  marlowe-cli run execute --marlowe-in-file collective-loan-$k.marlowe          \
                          --tx-in-marlowe "$TX_OLD#1"                           \
                          --tx-in-collateral $TX_ADA                            \
                          --tx-in $TX_ADA                                       \
                          --tx-in "$(find_role_token $ADMIN_ADDR $ADMIN_ROLE)"  \
                          --tx-in "$(find_role_token $ADMIN_ADDR $ROLE)"        \
                          --tx-out "$(make_role_txout $ADMIN_ADDR $ADMIN_ROLE)" \
                          --tx-out "$(make_role_txout $ADDR $ROLE)"             \
                          --marlowe-out-file collective-loan-$k1.marlowe        \
                          --change-address $ADMIN_ADDR                          \
                          --required-signer $ADMIN_SKEY                         \
                          --out-file /dev/null                                  \
                          --submit 600                                          \
  | sed -e 's/^TxId "\(.*\)"$/\1/'                                              \
  )
  echo 'TxId "'$TX_NEW'"'
  TX_OLD=$TX_NEW
  echo "Lender deposits liquidity tokens using their newly-received role token."
  marlowe-cli run prepare --deposit-account Role=$ADMIN_ROLE         \
                          --deposit-party Role=$ROLE                 \
                          --deposit-amount $AMOUNT                   \
                          --deposit-token $LIQUIDITY                 \
                          --invalid-before $NOW                      \
                          --invalid-hereafter $DEADLINE              \
                          --marlowe-file collective-loan-$k1.marlowe \
                          --out-file     collective-loan-$k2.marlowe 
  TX_ADA=$(find_ada $ADDR)
  TX_NEW=$(
  marlowe-cli run execute --marlowe-in-file collective-loan-$k1.marlowe  \
                          --tx-in-marlowe "$TX_OLD#1"                    \
                          --tx-in-collateral $TX_ADA                     \
                          --tx-in $TX_ADA                                \
                          --tx-in "$(find_token $ADDR $LIQUIDITY)"       \
                          --tx-in "$(find_role_token $ADDR $ROLE)"       \
                          --tx-out "$(make_role_txout $ADDR $ROLE)"      \
                          --marlowe-out-file collective-loan-$k2.marlowe \
                          --change-address $ADDR                         \
                          --required-signer $SKEY                        \
                          --out-file /dev/null                           \
                          --submit 600                                   \
  | sed -e 's/^TxId "\(.*\)"$/\1/'                                       \
  )
  echo 'TxId "'$TX_NEW'"'
  TX_OLD=$TX_NEW
  echo "Lender withdraws their funds from the payout address."
  TX_ADA=$(find_ada $ADDR)
  marlowe-cli run withdraw --marlowe-file collective-loan-$k2.marlowe \
                           --role-name $ROLE                          \
                           --tx-in-collateral $TX_ADA                 \
                           --tx-in $TX_ADA                            \
                           --tx-in $(find_role_token $ADDR $ROLE)     \
                           --tx-out "$(make_role_txout $ADDR $ROLE)"  \
                           --change-address $ADDR                     \
                           --required-signer $SKEY                    \
                           --out-file /dev/null                       \
                           --submit 600
  k=$k2
done


--- Lender redemption #1 ---
Lender 18 requests to deposit 2493804 liquidity tokens.
Administrator approves deposit and sends role token to lender.
TxId "ca43c3b64451dc577a9257c4a0ed8527ad4234293f6a2535ff2aa2b4719e06c9"
Lender deposits liquidity tokens using their newly-received role token.
Payment 1
  Acccount: "A"
  Payee: Party "LP1"
  Ada: 0.000000
  76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf."DjedUSD": 2589107
TxId "3be3ecfe79dbb6383aa1bcb12fc4d622413867f68859036e308b9f6eb349a1ce"
Lender withdraws their funds from the payout address.
TxId "bb9935408761c67c5b99d90bb41800bdb31fc4e16b708f2e54288a207fb11cd8"

--- Lender redemption #2 ---
Lender 20 requests to deposit 4076982 liquidity tokens.
Administrator approves deposit and sends role token to lender.
TxId "c539adba7b9913b72b822564ec242a6a7c009f3618c62527810805c0f3ba8bff"
Lender deposits liquidity tokens using their newly-received role token.
Payment 1
  Acccount: "A"
  Payee: Party "LP2"
  Ada: 0.000000
  76ea6a9aad

Now the contract accounts show that the contract only retains the liquidity tokens.

In [82]:
jq .state.accounts collective-loan-$k.marlowe | json2yaml

- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: A
    - currency_symbol: 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd
      token_name: Liquidity
  - 150000000
- - - role_token: A
    - currency_symbol: 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf
      token_name: DjedUSD
  - 1


View the UTxO at the script address.

In [83]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
a5e8b04a6ee73868ddf30e02e9cdd5506f9aa8e27b033db342dbe3bcbb51df15     1        2000000 lovelace + 150000000 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.4c6971756964697479 + 1 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf.446a6564555344 + TxOutDatumHash ScriptDataInAlonzoEra "00462e61c96777e7c4146446a8a29e8f65bacebc5e325afcf047399d59ae35f2"


There are no UTxOs at the payout address.

In [84]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ROLES_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


Defragment and then view the UTxOs at the lenders' addresses to see that they have their role tokens and liquidity tokens.

In [85]:
for j in `seq 1 $N_LENDERS`
do
  echo
  echo "Lender $j = ${LENDER_ADDR[$j]}"
  marlowe-cli util clean --required-signer ${LENDER_SKEY[$j]} --change-address ${LENDER_ADDR[$j]} --out-file /dev/null --submit 600
  cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address ${LENDER_ADDR[$j]}
done


Lender 1 = addr_test1qzenwj7elwatk3mmc368mwnv067fqnuk3x7u4nqw5dezg8wr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqf6k36p
TxId "7c22fd54a21f3657652fcf2b25c73b65ae173eb78990b1d3e279a681a9cc6db1"
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7c22fd54a21f3657652fcf2b25c73b65ae173eb78990b1d3e279a681a9cc6db1     0        19625434 lovelace + TxOutDatumNone
7c22fd54a21f3657652fcf2b25c73b65ae173eb78990b1d3e279a681a9cc6db1     1        2000000 lovelace + 5135878 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf.446a6564555344 + TxOutDatumNone
7c22fd54a21f3657652fcf2b25c73b65ae173eb78990b1d3e279a681a9cc6db1     2        2000000 lovelace + 1 9cb30bdd840cc8b31806c20eae6ed14614d32f10379547d605064e78.4c4433 + TxOutDatumNone
7c22fd54a21f3657652fcf2b25c73b65ae173eb78990b1d3e279a681a9cc6db1     3        2000000 lovelace + 1 9cb30bdd840cc8b31806c20eae6ed14614d32f10379547d60

### 8. Close the contract

The administrator receives back their initial Ada and liquidity tokens.

In [86]:
marlowe-cli run prepare --notify                                                \
                        --invalid-before $NOW                                   \
                        --invalid-hereafter $((ULTIMATE_DEADLINE - 1 * MINUTE)) \
                        --marlowe-file collective-loan-$k.marlowe               \
                        --out-file     collective-loan-$((1+k)).marlowe

Payment 1
  Acccount: "A"
  Payee: Party "A"
  Ada: 2.000000
Payment 2
  Acccount: "A"
  Payee: Party "A"
  Ada: 0.000000
  5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd."Liquidity": 150000000
Payment 3
  Acccount: "A"
  Payee: Party "A"
  Ada: 0.000000
  76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf."DjedUSD": 1


Submit the transactions.

In [87]:
TX_ADA=$(find_ada $ADMIN_ADDR)
marlowe-cli run execute --marlowe-in-file collective-loan-$k.marlowe          \
                        --tx-in-marlowe "$TX_OLD#1"                           \
                        --tx-in-collateral $TX_ADA                            \
                        --tx-in $TX_ADA                                       \
                        --tx-in "$(find_role_token $ADMIN_ADDR $ADMIN_ROLE)"  \
                        --tx-out "$(make_role_txout $ADMIN_ADDR $ADMIN_ROLE)" \
                        --marlowe-out-file collective-loan-$((k+1)).marlowe   \
                        --change-address $ADMIN_ADDR                          \
                        --required-signer $ADMIN_SKEY                         \
                        --out-file /dev/null                                  \
                        --submit 600      

TxId "e42d58869dce889c561fec062c47299be49877a98d404c6f1eea9808ca07f82f"


In [88]:
TX_ADA=$(find_ada $ADMIN_ADDR)
marlowe-cli run withdraw --marlowe-file collective-loan-$((k+1)).marlowe       \
                         --role-name $ADMIN_ROLE                               \
                         --tx-in-collateral $TX_ADA                            \
                         --tx-in $TX_ADA                                       \
                         --tx-in $(find_role_token $ADMIN_ADDR $ADMIN_ROLE)    \
                         --tx-out "$(make_role_txout $ADMIN_ADDR $ADMIN_ROLE)" \
                         --change-address $ADMIN_ADDR                          \
                         --required-signer $ADMIN_SKEY                         \
                         --out-file /dev/null                                  \
                         --submit 600

TxId "402a9b9ff54e541bbf9d25a64bf333b2c011152fddde8b28e0020e5c0097deaf"


There is no UTxO at the script address.

In [89]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


There are no UTxOs at the payout address.

In [90]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ROLES_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


Defragment and then view the UTxOs at the administrator's address.

In [91]:
marlowe-cli util clean --required-signer $ADMIN_SKEY --change-address $ADMIN_ADDR --out-file /dev/null --submit 600
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ADMIN_ADDR

TxId "51dc6cf65e37defa0369fa00d1388a39d4c985de7b597f3fc8e2c9ef14e82ae3"
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
51dc6cf65e37defa0369fa00d1388a39d4c985de7b597f3fc8e2c9ef14e82ae3     0        18523420359 lovelace + TxOutDatumNone
51dc6cf65e37defa0369fa00d1388a39d4c985de7b597f3fc8e2c9ef14e82ae3     1        2000000 lovelace + 150000000 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.4c6971756964697479 + TxOutDatumNone
51dc6cf65e37defa0369fa00d1388a39d4c985de7b597f3fc8e2c9ef14e82ae3     2        2000000 lovelace + 1 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf.446a6564555344 + TxOutDatumNone
51dc6cf65e37defa0369fa00d1388a39d4c985de7b597f3fc8e2c9ef14e82ae3     3        2000000 lovelace + 1 9cb30bdd840cc8b31806c20eae6ed14614d32f10379547d605064e78.41 + TxOutDatumNone


## Clean Up

Use Daedalus to send all of the lender and borrower tokens to the administrator.

In [92]:
echo $ADMIN_ADDR
false

addr_test1vrssw4edcts00kk6lp7p5n64666m23tpprqaarmdwkaq69gfvqnpz


: 1

Burn the Djed.

In [93]:
marlowe-cli util mint --required-signer $ADMIN_SKEY \
                      --change-address $ADMIN_ADDR  \
                      --expires 1000000011          \
                      --count -$TOTAL_DJED          \
                      --out-file /dev/null          \
                      --submit 600                  \
                      $DJED_NAME

PolicyID "76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf"


Burn the liquidity tokens.

In [94]:
marlowe-cli util mint --required-signer $ADMIN_SKEY \
                      --change-address $ADMIN_ADDR  \
                      --expires 1000000012          \
                      --count -$POOL_SIZE           \
                      --out-file /dev/null          \
                      --submit 600                  \
                      $LIQUIDITY_NAME

PolicyID "5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd"


Burn the role tokens.

In [95]:
marlowe-cli util mint --required-signer $ADMIN_SKEY \
                      --change-address $ADMIN_ADDR  \
                      --expires 1000000013          \
                      --count -1                    \
                      --out-file /dev/null          \
                      --submit 600                  \
                      $ADMIN_ROLE                   \
                      ${LENDER_DEPOSIT_ROLE[@]}     \
                      ${LENDER_PAYMENT_ROLE[@]}     \
                      ${BORROWER_DEPOSIT_ROLE[@]}   \
                      ${BORROWER_PAYMENT_ROLE[@]}

PolicyID "9cb30bdd840cc8b31806c20eae6ed14614d32f10379547d605064e78"


Check that all of the burning was successful.

In [96]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ADMIN_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7ce391792e2fd04c2bc53d416f28bdb6bf1bf8af371a0cd76a4309e7672dd50b     0        18547732330 lovelace + TxOutDatumNone
